In [ ]:
import psi4
import numpy as np
import datetime
from scipy import linalg as splinalg

In [ ]:
psi4.set_memory('500 MB')
numpy_memory = 2
psi4.core.set_output_file('output.dat', False)
basis = 'sto-3g'
psi4.set_options({'basis': basis,
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})
mol = psi4.geometry("""
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")
SCF_E_psi = psi4.energy('scf')
psi4.core.clean()
print(f"The Hartree-Fock ground state energy of the water is: {SCF_E_psi:.6f} Eh")

In [ ]:
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))
ndocc = wfn.nalpha()    #Number of doubly occupied orbitals
nbf = wfn.basisset().nbf()
mints = psi4.core.MintsHelper(wfn.basisset())   #Molecular integrals object
S_matrix = mints.ao_overlap()
S = np.asarray(S_matrix)
#Testing for orthonormality
def isBasisOrthonormal(S):
    size_S = S.shape[0] 
    identity_matrix = np.eye(size_S) 
    orthonormal_check = np.allclose(S, identity_matrix)
    print(F'Q:(T/F) The AO basis is orthonormal? A: {orthonormal_check}')
    return orthonormal_check
A = splinalg.sqrtm(np.linalg.inv(S))    #A=S^-1/2
S_p = A.dot(S.dot(A))   #S'=ASA
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V
F_p = A.dot(H.dot(A))   #Initial guess of F = Core hamiltonian
vals, vecs = np.linalg.eigh(F_p)    #Solve HFR to get C
C=A.dot(vecs)
C_occ = C[:, :ndocc]
D = np.einsum('pi,qi->pq',C_occ ,C_occ )    #Using einsum to compute D, J, K
I = np.asarray(mints.ao_eri())
J = np.einsum('rs,pqrs->pq',D,I )
K = np.einsum('rq,pqrs->ps',D,I )
F = H + 2*J - K
isBasisOrthonormal(S_p)

In [ ]:
start_time = datetime.datetime.now()
# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()
# ==> SCF Iterations <==
SCF_E = 0.0
E_old = 0.0
MAXITER = 40
E_conv = 1.0e-8
print('==> Starting SCF Iterations <==\n')
# Begin Iterations
for scf_iter in range(1, MAXITER + 1):
    I = np.asarray(mints.ao_eri())
    J = np.einsum('rs,pqrs->pq',D,I )
    K = np.einsum('rq,pqrs->ps',D,I )
    F = H + 2*J - K    
    SCF_E = E_nuc + np.einsum('pq->',(H+F)*(D))
    print(F'SCF Iteration {scf_iter}: Energy = {SCF_E:.8f} dE = {SCF_E - E_old:.8f}')
    if (abs(SCF_E - E_old) < E_conv):
        break
    E_old = SCF_E
    F_p=A.dot(F.dot(A))
    vals, vecs = np.linalg.eigh(F_p)
    C=A.dot(vecs)
    C_occ = C[:, :ndocc]
    D = np.einsum('pi,qi->pq',C_occ ,C_occ ) 
    if (scf_iter == MAXITER):
        psi4.core.clean()
        raise Exception("Maximum number of SCF iterations exceeded.")
# Post iterations
print('\nSCF converged.')
print(F'Final RHF Energy: {SCF_E:.6f} [Eh]')
end_time = datetime.datetime.now()
print("This took %f seconds" %(end_time-start_time).total_seconds())